# Introduction and Background

According to the United States Census Bureau, Columbus becomes the14th largest city in the U.S. and is the second-largest city in the Midwest. Since the 2010 census, Columbus has grown 9.3 percent, from 787,033 residents in 2010 to 860,090 in 2016. Between 2011 and 2016, private employment in Ohio increased by 450,000 jobs, far outperforming the regional average of 261,000 jobs. City officials say they expect more than a million people to move to Central Ohio over the course of the next 20 to 25 years. The rapid growth presents big opportunities to investors from all areas. Along with the population expansion, Columbus is rapidly gaining a reputation as one of America's up-and-coming culinary destinations. Its restaurant scene is creating a buzz with its creativity, diversity and one-of-a-kind experiences. Food & Wine recognized Columbus as one of the best culinary destinations in the U.S.
An investor from Japan would like to take the advantage of the rapid growth of Columbus and invest in restaurant industry, specifically Japanese restaurant, in Columbus and its surrounding cities. He would like to make data-driven decisions on the new locations that are most suitable for the new restaurants. This project will examine and visualize current restaurants in Franklin County.


## Questions to be addressed:



1. Which is the distribution of restaurants in each city in Franklin County?<br />
2. Which cities have potential market for Japanese restaurant?

## Data


For this project we will use the following data:<br />
1. Ohio data that contains list of cities and coordinations<br />
    A list of cities: https://en.wikipedia.org/wiki/List_of_cities_in_Ohio<br />
    Coordinates: https://raw.githubusercontent.com/yywang118/Coursera-Capstone/master/OHIO_location.csv<br />
2.	Restaurants in each city of Franklin County; <br />
    Data source : Fousquare API<br />
    Description : By using this api we will get all the venues in each city. 


## Approach
1. Collect the Ohio city data from https://en.wikipedia.org/wiki/List_of_cities_in_Ohio
2. Collect the latitude and longitude for each city from https://raw.githubusercontent.com/yywang118/Coursera-Capstone/master/OHIO_location.csv
3. Using FourSquare API we will find all venues for each city.
4. Filter out all venues that are Restaurants.
5. Find rating, tips and like count for each Indian Restaurants using FourSquare API.
6. Using rating for each restaurant, we will sort that data.
7. Visualize the Ranking of neighborhoods using folium library(python)
